# Trabalho Parte 2

Nome: Murilo Guidetti Andrietta             Ra: 147472  
Nome: Naomi Takemoto                        Ra: 184849  


## Modelo Conceitual
![Imagem Modelo Conceitual](modeloConceitual.jpeg).


## Modelo Lógico
Sample(\_ID_Sample_, Clump_Thickness, Uniformity_of_Cell_Size, Uniformity_of_Cell_Shape, Marginal_Adhesion, Single_Epithelial_Cell_Size, Bare_Nuclei, Bland_Chromatin, Normal_Nucleoli, Mitoses, ID_Dianostic, ID_Patient);  
    CHE: ID_Dianostic (Diagnostic: ID_Diagnostic)  
    CHE: ID_Patient (Patient: ID);  

Diagnostic (\_ID_Diagnostic_, Result);  

Patient (\_ID_, Name, Sex, Age);  


Patient-Previous_Diseases(\_ID_Patient_, \_ID_Previous_Diseases_);  
    CHE: ID_Previous_Diseases (Previous_Diseases: ID)  
    CHE: ID_Patient (Patient: ID);  


Patient-Symptoms (\_ID_Patient_, _\ID_Symptoms_);  
ID_Symptoms (Symptoms: ID)  
    CHE: ID_Patient (Patient: ID);  


Previous_Diseases (\_ID_, Name, Chronic);  


Symptoms(ID, Name, Frequency_per_year, Intensity);  

## Algumas considerações:
A aplicação proposta pela dupla busca diagnosticar pacientes que fazem exames (geram samples) utilizando a técnica de machine learnig. Para que seja possível treinar um algoritmo deste tipo são necessários dados de amostras das quais o algoritmo consegue "aprender", nesta etapa criamos as relações para representar cada uma das classes descritas no modelo conceitual. No entanto, os dados carregados a partir do documentoo csv contém apenas informaçãoes referentes à sample, que contém informações a serem usadas no processo de treinamento do algoritmo de aprendizagem automática.  
A ideia é, na sua totalidade, o sistema relaciona um paciente (com seus respectivos atributos) e seu conjunto de amostras provenientes de exames.

In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Diagnostic;
DROP TABLE IF EXISTS Sample;
DROP TABLE IF EXISTS Patient;
DROP TABLE IF EXISTS Previous_Diseases;
DROP TABLE IF EXISTS Symptoms;
DROP TABLE IF EXISTS Patient_Previous_Diseases;
DROP TABLE IF EXISTS Patient_Symptoms;


CREATE TABLE Diagnostic
(
    ID_Diagnostic INTEGER, 
    Result VARCHAR(1) NOT NULL,
    PRIMARY KEY (ID_Diagnostic)
);



CREATE TABLE Sample
(
    ID_Sample INTEGER NOT NULL,
    Clump_Thickness INTEGER NOT NULL,
    Uniformity_of_Cell_Size INTEGER NOT NULL,
    Uniformity_of_Cell_Shape INTEGER NOT NULL,
    Marginal_Adhesion INTEGER NOT NULL,
    Single_Epithelial_Cell_Size INTEGER NOT NULL,
    Bare_Nuclei INTEGER NOT NULL,
    Bland_Chromatin INTEGER NOT NULL,
    Normal_Nucleoli INTEGER NOT NULL,
    Mitoses INTEGER NOT NULL,
    ID_Diagnostic INTEGER --NOT NULL,
   -- ID_Patient INTEGER, -- NOT NULL,
    
--     PRIMARY KEY(ID_Sample),
)
AS SELECT
ID_Sample,
Clump_Thickness,
Uniformity_of_Cell_Size,
Uniformity_of_Cell_Shape,
Marginal_Adhesion,
Single_Epithelial_Cell_Size,
Bare_Nuclei,
Bland_Chromatin,
Normal_Nucleoli,
Mitoses,
Diagnostic

FROM CSVREAD('../data/breast-cancer-wisconsin.csv');


CREATE TABLE Patient
(
    ID INTEGER NOT NULL,
    Name VARCHAR(500) NOT NULL,
    Sex VARCHAR(1) NOT NULL, 
    Age INTEGER NOT NULL,
    Sample INTEGER,
    
    PRIMARY KEY(ID)
);

CREATE TABLE Previous_Diseases
(
    ID INTEGER NOT NULL,
    Name VARCHAR(500) NULL, 
    Chronic BOOLEAN NOT NULL,
    
    PRIMARY KEY(ID)
);

CREATE TABLE Symptoms
(
    ID INTEGER NOT NULL, 
    Name VARCHAR(500) NOT NULL, 
    Frequency_per_year INTEGER NOT NULL, 
    Intensity INTEGER NOT NULL,
    
    PRIMARY KEY(ID)
);

CREATE TABLE Patient_Previous_Diseases
(
    ID_Patient INTEGER NOT NULL, 
    ID_Previous_Diseases INTEGER NOT NULL,
    
    PRIMARY KEY (ID_Patient, ID_Previous_Diseases),
    
    FOREIGN KEY (ID_Patient) 
       REFERENCES Patient(ID)
       ON DELETE NO ACTION
       ON UPDATE NO ACTION,
    
    FOREIGN KEY (ID_Previous_Diseases) 
       REFERENCES Previous_Diseases(ID)
       ON DELETE NO ACTION
       ON UPDATE NO ACTION
);

CREATE TABLE Patient_Symptoms
(
    ID_Patient INTEGER NOT NULL, 
    ID_Symptoms INTEGER NOT NULL,
    
    PRIMARY KEY (ID_Patient, ID_Symptoms),
    
    FOREIGN KEY (ID_Patient) 
       REFERENCES Patient(ID)
       ON DELETE NO ACTION
       ON UPDATE NO ACTION,
    
    FOREIGN KEY (ID_Symptoms) 
       REFERENCES Symptoms(ID)
       ON DELETE NO ACTION
       ON UPDATE NO ACTION
);



## Relacionando alguns pacientes com uma sintomas, amostras e histórico

In [3]:
INSERT INTO Patient VALUES (1,'Cremilda', 'F', 90, 1016277);
INSERT INTO Previous_Diseases VALUES(1,'Diabetes', True);
INSERT INTO Patient_Previous_Diseases VALUES (1, 1);
INSERT INTO Symptoms VALUES (1,'Dor de Cabeça',10, 2);
INSERT INTO Patient_Symptoms VALUES (1,1);
INSERT INTO Patient VALUES (2,'Pessoa', 'F', 90, 1002945);
 

In [4]:
SELECT * 
FROM Patient, Sample
WHERE Patient.sample = Sample.ID_sample;

In [5]:
SELECT * FROM Sample;

## Número de amostras malígnas

In [6]:
DROP TABLE IF EXISTS AmostrasMalignas;

CREATE VIEW AmostrasMalignas AS
SELECT COUNT(*) Malignos
FROM Sample s
WHERE s.Id_Diagnostic = 4;

SELECT * FROM AmostrasMalignas;

239

## Número de Amostras Benígnas

In [7]:
DROP TABLE IF EXISTS AmostrasBenignas;

CREATE VIEW AmostrasBenignas AS
SELECT COUNT(*) Benignos
FROM Sample s
WHERE s.Id_Diagnostic = 2;

SELECT * FROM AmostrasBenignas;

444

## Número de amostras malígnas que apresentam anormalias no tamanho e forma das células.

In [8]:
SELECT COUNT(*) M
FROM Sample, AmostrasMalignas
WHERE (Uniformity_of_Cell_Size >= 7) AND (Uniformity_of_Cell_Shape >= 7) AND (Sample.Id_Diagnostic = 4);

98

## Número de amostras malígnas que apresentam anormalias no núcleo.

In [9]:
SELECT COUNT(*) M
FROM Sample, AmostrasMalignas
WHERE (Normal_Nucleoli >= 7) AND (Bare_Nuclei >= 7) AND (Sample.Id_Diagnostic = 4);

70

## Número de amostras benígnas que possuem baixo valor de anomalia na forma e tamanho da célula.

In [10]:
SELECT COUNT(*) M
FROM Sample, AmostrasMalignas
WHERE (Uniformity_of_Cell_Size <= 3) AND (Uniformity_of_Cell_Shape <= 3) AND (Sample.Id_Diagnostic = 2);

424

## Número de amostras que possuem baixo valor de anomalia no núcleo.

In [11]:
SELECT COUNT(*) M
FROM Sample, AmostrasMalignas
WHERE (Normal_Nucleoli <= 3) AND (Bare_Nuclei <= 3) AND (Sample.Id_Diagnostic = 2);

415

## Número de amostras malígnas que possuem média de todos os atributos maiores que 5.

In [12]:
DROP TABLE IF EXISTS Attributes_Average;

CREATE VIEW Attributes_Average AS
SELECT Id_Sample, ((Clump_Thickness + Uniformity_of_Cell_Size + Uniformity_of_Cell_Shape + Marginal_Adhesion + Single_Epithelial_Cell_Size +
       Bare_Nuclei + Bland_Chromatin + Normal_Nucleoli + Mitoses)/9) AS Media FROM Sample;
       
SELECT COUNT(*) FROM Attributes_Average AA, Sample S WHERE (AA.Id_Sample = S.Id_Sample) AND (AA.Media > 5) AND (S.Id_Diagnostic = 4);

126

## Número de amostras benígnas que possuem média de todos os atributos menores que 5.

In [13]:
SELECT COUNT(*) FROM Attributes_Average AA, Sample S WHERE (AA.Id_Sample = S.Id_Sample) AND (AA.Media < 5) AND (S.Id_Diagnostic = 2);

559